In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

### データ読み込み

In [2]:
# ユーザー特徴量を読み込み
user_features = np.load('user_features_standard.npy')
print(f"ユーザー特徴量の形状: {user_features.shape}")

# ユーザーマッピングを読み込み
with open('user_mapping_standard.json', 'r') as f:
    user_mapping = json.load(f)
print(f"ユーザーマッピング数: {len(user_mapping)}")

# BigQueryデータから年齢情報を取得
df = pd.read_csv('bigquery_results.csv')

# 各ユーザーの年齢を取得（最も頻度の高い年齢を使用）
user_age = df.groupby('common_id')['age'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0])

print(f"年齢分布:")
age_counts = user_age.value_counts().sort_index()
for age, count in age_counts.items():
    print(f"  {age}歳: {count}人")

# ユーザー特徴量と年齢ラベルを対応付け
X = []
y = []
user_ids = []

for i, (user_id, features) in enumerate(zip(user_mapping.keys(), user_features)):
    if user_id in user_age.index:
        X.append(features)
        y.append(user_age[user_id])
        user_ids.append(user_id)

X = np.array(X)
y = np.array(y)

print(f"\n=== データセットの準備 ===")
print(f"特徴量の形状: {X.shape}")
print(f"ラベルの形状: {y.shape}")
print(f"年齢の統計情報:")
print(f"  平均年齢: {np.mean(y):.2f}歳")
print(f"  標準偏差: {np.std(y):.2f}歳")
print(f"  最小年齢: {np.min(y)}歳")
print(f"  最大年齢: {np.max(y)}歳")
print(f"  年齢範囲: {np.max(y) - np.min(y)}歳")

ユーザー特徴量の形状: (56, 20)
ユーザーマッピング数: 56
年齢分布:
  20歳: 14人
  30歳: 6人
  40歳: 11人
  50歳: 14人
  60歳: 11人

=== データセットの準備 ===
特徴量の形状: (56, 20)
ラベルの形状: (56,)
年齢の統計情報:
  平均年齢: 40.36歳
  標準偏差: 14.63歳
  最小年齢: 20歳
  最大年齢: 60歳
  年齢範囲: 40歳


In [3]:
# データを訓練・テストに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"訓練データ: {X_train.shape[0]}サンプル")
print(f"テストデータ: {X_test.shape[0]}サンプル")

訓練データ: 39サンプル
テストデータ: 17サンプル


### 学習

In [8]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.1, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
}

results = {}

for name, model in models.items():
    print(f"\n--- {name} ---")
    
    # 訓練
    model.fit(X_train, y_train)
    
    # 予測
    y_pred = model.predict(X_test)
    
    # 評価指標を計算
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    # 予測結果の詳細
    print(f"予測結果の統計:")
    print(f"  実際の年齢範囲: {np.min(y_test)}-{np.max(y_test)}歳")
    print(f"  予測年齢範囲: {np.min(y_pred):.1f}-{np.max(y_pred):.1f}歳")
    print(f"  予測誤差の平均: {np.mean(np.abs(y_test - y_pred)):.2f}歳")
    print(f"  予測誤差の標準偏差: {np.std(np.abs(y_test - y_pred)):.2f}歳")


--- Linear Regression ---
MAE: 15840653.6746
RMSE: 64835439.3474
R²: -21313162850229.4609
予測結果の統計:
  実際の年齢範囲: 20-60歳
  予測年齢範囲: -1415300.1-267319188.7歳
  予測誤差の平均: 15840653.67歳
  予測誤差の標準偏差: 62870564.55歳

--- Ridge Regression ---
MAE: 12.9171
RMSE: 15.1394
R²: -0.1621
予測結果の統計:
  実際の年齢範囲: 20-60歳
  予測年齢範囲: 40.5-47.5歳
  予測誤差の平均: 12.92歳
  予測誤差の標準偏差: 7.90歳

--- Lasso Regression ---
MAE: 13.4790
RMSE: 16.3422
R²: -0.3541
予測結果の統計:
  実際の年齢範囲: 20-60歳
  予測年齢範囲: 39.5-58.3歳
  予測誤差の平均: 13.48歳
  予測誤差の標準偏差: 9.24歳

--- Random Forest ---
MAE: 12.0765
RMSE: 14.0901
R²: -0.0066
予測結果の統計:
  実際の年齢範囲: 20-60歳
  予測年齢範囲: 31.8-50.5歳
  予測誤差の平均: 12.08歳
  予測誤差の標準偏差: 7.26歳


In [11]:
# Random Forest
print(y_test)
print(y_pred)

[50 40 20 60 20 20 40 20 30 60 30 40 50 50 20 50 30]
[34.9 42.8 50.5 44.2 32.6 35.8 40.  33.9 43.8 38.6 46.  44.1 45.7 38.3
 31.8 45.2 40.9]


### クロスバリデーション

In [12]:
# 回帰モデルを定義
from sklearn.model_selection import KFold

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.1, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\n--- {name} クロスバリデーション ---")
    
    # MAEでクロスバリデーション
    mae_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    mae_scores = -mae_scores  # 負の値を正の値に変換
    
    # MSEでクロスバリデーション
    mse_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    mse_scores = -mse_scores  # 負の値を正の値に変換
    
    print(f"MAE - 各フォールド: {mae_scores}")
    print(f"MAE - 平均: {mae_scores.mean():.4f} (+/- {mae_scores.std() * 2:.4f})")
    print(f"MSE - 各フォールド: {mse_scores}")
    print(f"MSE - 平均: {mse_scores.mean():.4f} (+/- {mse_scores.std() * 2:.4f})")


--- Linear Regression クロスバリデーション ---
MAE - 各フォールド: [1.10119619e+07 3.50125220e+03 3.69728852e+05 5.48122021e+09
 7.73045898e+03]
MAE - 平均: 1098522626.4347 (+/- 4382705701.5695)
MSE - 各フォールド: [1.45318150e+15 4.56600495e+07 8.49239689e+11 3.30480771e+20
 3.70406099e+08]
MSE - 平均: 66096445031804223488.0000 (+/- 264384326098460606464.0000)

--- Ridge Regression クロスバリデーション ---
MAE - 各フォールド: [13.25420677 13.22653853 13.824854   11.50666199 14.2066766 ]
MAE - 平均: 13.2038 (+/- 1.8493)
MSE - 各フォールド: [256.47614765 221.71989884 234.78920383 208.5409783  253.39610167]
MSE - 平均: 234.9845 (+/- 36.6183)

--- Lasso Regression クロスバリデーション ---
MAE - 各フォールド: [13.89190288 13.69380924 13.75900898 11.64282677 14.7168532 ]
MAE - 平均: 13.5409 (+/- 2.0356)
MSE - 各フォールド: [299.11461076 247.77555678 231.41600538 209.75872344 275.54653352]
MSE - 平均: 252.7223 (+/- 63.2297)

--- Random Forest クロスバリデーション ---
MAE - 各フォールド: [13.45833333 11.89090909 11.33636364 12.1        13.76363636]
MAE - 平均: 12.5098 (+/- 1.8761)
MSE 